In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 5;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;
rc_params["axes.spines.right"] = false
rc_params["axes.spines.top"] = false

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
data_info_dict = load("../figure_data_info.jld2")

data_info_all = []
data_info_tog = []
job_names = []
for key in keys(data_info_dict)
    append!(data_info_all, data_info_dict[key])
    append!(data_info_tog, [data_info_dict[key]])
    append!(job_names, [String(key)])
end

In [ ]:
analyzer = "chuyu"

In [ ]:
using Printf
function plot_mean_density(bins, y_tog; plot_color="k", shade_color="gray", line_color="k", plot_median = true)
    
    n_bins = length(bins)
    x = (bins[1:end-1] + bins[2:end])/2
    y = mean(y_tog)
    ymax = y + std(y_tog)
    ymin = y - std(y_tog);
    
    cumsum_y_mean = cumsum(mean(y_tog)*(maximum(bins) - minimum(bins))/(n_bins-1))

    x_median = numpy.round(numpy.interp(0.5, cumsum_y_mean, x), 2)
    x_14 = numpy.round(numpy.interp(0.25, cumsum_y_mean, x), 2)
    x_34 = numpy.round(numpy.interp(0.75, cumsum_y_mean, x), 2)
    
    
    plot(x, y, color= plot_color)
    fill_between(x, ymax, ymin, color=shade_color, alpha=0.3)
    
    # println(minimum(ymin) , " ", maximum(ymax))
    if plot_median
        axvline(x_median, color=line_color, label= @sprintf("%.2f",x_median), linestyle="dashed")
        # axvline(x_median, color=line_color, label= "$x_median ($x_14 - $x_34)")
    end
    
    
    return ymin, ymax
end

In [ ]:
job_names_ordered = [
 "corner_cue_rotation_whole"
 "landmark_removal"
 "boundary_morphing"
    
    
 "job_retangular_half_circle_rotation"

    
 "hedgehog_rotation_bottom_out"
"hedgehog_landmark_removal"
 "hedgehog_circle"
 "corner_cue_circle"
    
]


In [ ]:
job_names_figure = [
 "Chamber rotation"
 "Landmark removal"
 "Wall morphing"
    
 "Wall rotation"
    
 "No change"
 "Landmark removal"
 "Wall morphing"
 "Wall morphing + \n Landmark removal"
]


In [ ]:
fig = figure(figsize=(6,0.3))
y_min_all = []
y_max_all = []
for (which_job, job_name) in enumerate(job_names_ordered)
if job_name != "boundary_morphing"
    global compare_map_file_name = "compare_map_results_matchangle.h5"
else
    global compare_map_file_name = "compare_map_results_matchangle.h5"
end
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[which_job]
    

println(job_name)

n_bins = 31

corr_trials_all_tog = []
corr_pv_all_normalized_tog = []
peak_shift_mm_tog = []
    
corr_trials_all_exp = []
corr_pv_all_normalized_exp = []
peak_shift_mm_exp = []
    
corr_trials_all_tog_control = []
corr_pv_all_normalized_tog_control = []
peak_shift_mm_tog_control = []
    
corr_trials_all_exp_control = []
corr_pv_all_normalized_exp_control = []
peak_shift_mm_exp_control = []


for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    info_filename = joinpath(data_path(ds_save_cy_1), compare_map_file_name)
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized)]
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_earlylate = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all_earlylate = HDF5.readmmap(file["corr_earlylate_all"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_dFF"])
    corr_pv_all_normalized_earlylate = pv_map_normalized_earlylate[isfinite.(pv_map_normalized_earlylate)]
    peak_shift_mm_earlylate = HDF5.readmmap(file["peak_shift_early_late_mm"])
    close(file)
        

    println(length(tel_place_cell_index))
    println(sum(isnan.(corr_trials_all[tel_place_cell_index])))
    
end




end

# match angle

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure6/summary/match_angle"
mkpath(fig_path)

In [ ]:
fig = figure(figsize=(6,0.3))
y_min_all = []
y_max_all = []
for (which_job, job_name) in enumerate(job_names_ordered)
if job_name != "boundary_morphing"
    global compare_map_file_name = "compare_map_results_matchangle.h5"
else
    global compare_map_file_name = "compare_map_results_matchangle.h5"
end
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[which_job]
    



n_bins = 31

corr_trials_all_tog = []
corr_pv_all_normalized_tog = []
peak_shift_mm_tog = []
    
corr_trials_all_exp = []
corr_pv_all_normalized_exp = []
peak_shift_mm_exp = []
    
corr_trials_all_tog_control = []
corr_pv_all_normalized_tog_control = []
peak_shift_mm_tog_control = []
    
corr_trials_all_exp_control = []
corr_pv_all_normalized_exp_control = []
peak_shift_mm_exp_control = []


for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    info_filename = joinpath(data_path(ds_save_cy_1), compare_map_file_name)
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized)]
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_earlylate = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all_earlylate = HDF5.readmmap(file["corr_earlylate_all"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_dFF"])
    corr_pv_all_normalized_earlylate = pv_map_normalized_earlylate[isfinite.(pv_map_normalized_earlylate)]
    peak_shift_mm_earlylate = HDF5.readmmap(file["peak_shift_early_late_mm"])
    close(file)
        
    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]
    corr_pv_all_normalized_earlylate = pv_map_normalized_earlylate[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]

    density_corr_trials, bins = numpy.histogram(corr_trials_all[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)], bins= numpy.linspace(0, 50,n_bins), density=true)
 
    append!(corr_trials_all_tog, [density_corr_trials])
    append!(corr_pv_all_normalized_tog, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog, [density_shift])
        
        
    append!(corr_trials_all_exp, [corr_trials_all[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp, [pv_map_normalized])
    append!(peak_shift_mm_exp, [peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)]])
 

        
    density_corr_trials, bins = numpy.histogram(corr_trials_all_earlylate[tel_place_cell_index][isfinite.(corr_trials_all[tel_place_cell_index])], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized_earlylate, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)], bins= numpy.linspace(0, 50,n_bins), density=true)

    append!(corr_trials_all_tog_control, [density_corr_trials])
    append!(corr_pv_all_normalized_tog_control, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog_control, [density_shift])
        
        
    append!(corr_trials_all_exp_control, [corr_trials_all_earlylate[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp_control, [pv_map_normalized_earlylate])
    append!(peak_shift_mm_exp_control, [peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)]])
 

    
    
end





subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(-1,1,n_bins)
y_tog = corr_trials_all_tog_control
y_min, y_max = plot_mean_density(bins, y_tog)
append!(y_min_all, y_min)    
append!(y_max_all, y_max)  


subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(-1,1,n_bins)
y_tog = corr_trials_all_tog
y_min, y_max = plot_mean_density(bins, y_tog; plot_color="r", shade_color="r", line_color="r")
append!(y_min_all, y_min)    
append!(y_max_all, y_max)      

xlabel("PF correlation", labelpad = -0.5)
xlim(-1,1)
    
title(job_name, rotation = 30, fontsize = 5)
if which_job == 1
    ylabel("Density")
end
    
if which_job != 1
    yticks([])
end
 
x = reduce(vcat, corr_trials_all_exp)
y = reduce(vcat, corr_trials_all_exp_control)

valid_neuron = isfinite.(x) .* isfinite.(y)
_,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")
siginficance = "n.s."
if p_value < 0.01
    siginficance = "*"
end
if p_value < 1e-3
    siginficance = "**"
end
if p_value < 1e-5
    siginficance = "***"
end
println(p_value)



end

for which_job in 1:length(job_names_ordered)
    subplot(1,length(job_names_ordered),which_job)
    ylim(numpy.min([y_min_all]),numpy.max([y_max_all]))
end

fig.savefig(joinpath(fig_path, "PF_correlation.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);

In [ ]:
fig = figure(figsize=(6,0.3))
y_min_all = []
y_max_all = []
for (which_job, job_name) in enumerate(job_names_ordered)
    
if job_name != "boundary_morphing"
    global compare_map_file_name = "compare_map_results_matchangle.h5"
else
    global compare_map_file_name = "compare_map_results_matchangle.h5"
end
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[which_job]
    



n_bins = 31

corr_trials_all_tog = []
corr_pv_all_normalized_tog = []
peak_shift_mm_tog = []
    
corr_trials_all_exp = []
corr_pv_all_normalized_exp = []
peak_shift_mm_exp = []
    
corr_trials_all_tog_control = []
corr_pv_all_normalized_tog_control = []
peak_shift_mm_tog_control = []
    
corr_trials_all_exp_control = []
corr_pv_all_normalized_exp_control = []
peak_shift_mm_exp_control = []


for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    info_filename = joinpath(data_path(ds_save_cy_1), compare_map_file_name)
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_earlylate = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all_earlylate = HDF5.readmmap(file["corr_earlylate_all"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm_earlylate = HDF5.readmmap(file["peak_shift_early_late_mm"])
    close(file)
        
    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]
    corr_pv_all_normalized_earlylate = pv_map_normalized_earlylate[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]

    density_corr_trials, bins = numpy.histogram(corr_trials_all[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)], bins= numpy.linspace(0, 50,n_bins), density=true)
 
    append!(corr_trials_all_tog, [density_corr_trials])
    append!(corr_pv_all_normalized_tog, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog, [density_shift])
        
        
    append!(corr_trials_all_exp, [corr_trials_all[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp, [pv_map_normalized])
    append!(peak_shift_mm_exp, [peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)]])
 

        
    density_corr_trials, bins = numpy.histogram(corr_trials_all_earlylate[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized_earlylate, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)], bins= numpy.linspace(0, 50,n_bins), density=true)

    append!(corr_trials_all_tog_control, [density_corr_trials])
    append!(corr_pv_all_normalized_tog_control, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog_control, [density_shift])
        
        
    append!(corr_trials_all_exp_control, [corr_trials_all_earlylate[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp_control, [pv_map_normalized_earlylate])
    append!(peak_shift_mm_exp_control, [peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)]])
 

    
    
end





subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(-1,1,n_bins)
y_tog = corr_pv_all_normalized_tog_control
y_min, y_max = plot_mean_density(bins, y_tog)
append!(y_min_all, y_min)    
append!(y_max_all, y_max)  



subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(-1,1,n_bins)
y_tog = corr_pv_all_normalized_tog
y_min, y_max = plot_mean_density(bins, y_tog; plot_color="r", shade_color="r", line_color="r")
append!(y_min_all, y_min)    
append!(y_max_all, y_max)  
    
    
xlabel("PV correlation", labelpad=-0.5)
xlim(-1,1)
    
title(" ", pad=20, rotation = 30)
if which_job == 1
    ylabel("Density")
end
    
if which_job != 1
    yticks([])
end

    
    
x = reduce(vcat, corr_pv_all_normalized_exp)
y = reduce(vcat, corr_pv_all_normalized_exp_control)

valid_neuron = isfinite.(x) .* isfinite.(y)
_,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")
siginficance = "n.s."
if p_value < 0.01
    siginficance = "*"
end
if p_value < 1e-3
    siginficance = "**"
end
if p_value < 1e-5
    siginficance = "***"
end
println(p_value)



end

for which_job in 1:length(job_names_ordered)
    subplot(1,length(job_names_ordered),which_job)
    ylim(numpy.min([y_min_all]),numpy.max([y_max_all]))
end
fig.savefig(joinpath(fig_path, "PV_correlation.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);

In [ ]:
y_min_all = []
y_max_all = []

fig = figure(figsize=(6,0.3))

for (which_job, job_name) in enumerate(job_names_ordered)
if job_name != "boundary_morphing"
    global compare_map_file_name = "compare_map_results_matchangle.h5"
else
    global compare_map_file_name = "compare_map_results_matchangle.h5"
end
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[which_job]
    



n_bins = 31

corr_trials_all_tog = []
corr_pv_all_normalized_tog = []
peak_shift_mm_tog = []
    
corr_trials_all_exp = []
corr_pv_all_normalized_exp = []
peak_shift_mm_exp = []
    
corr_trials_all_tog_control = []
corr_pv_all_normalized_tog_control = []
peak_shift_mm_tog_control = []
    
corr_trials_all_exp_control = []
corr_pv_all_normalized_exp_control = []
peak_shift_mm_exp_control = []


for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    info_filename = joinpath(data_path(ds_save_cy_1), compare_map_file_name)
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_earlylate = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all_earlylate = HDF5.readmmap(file["corr_earlylate_all"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm_earlylate = HDF5.readmmap(file["peak_shift_early_late_mm"])
    close(file)
        

    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]
    corr_pv_all_normalized_earlylate = pv_map_normalized_earlylate[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]

    density_corr_trials, bins = numpy.histogram(corr_trials_all[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)], bins= numpy.linspace(0, 50,n_bins), density=true)
 
    append!(corr_trials_all_tog, [density_corr_trials])
    append!(corr_pv_all_normalized_tog, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog, [density_shift])
        
        
    append!(corr_trials_all_exp, [corr_trials_all[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp, [pv_map_normalized])
    append!(peak_shift_mm_exp, [peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)]])
 

        
    density_corr_trials, bins = numpy.histogram(corr_trials_all_earlylate[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized_earlylate, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)], bins= numpy.linspace(0, 50,n_bins), density=true)

    append!(corr_trials_all_tog_control, [density_corr_trials])
    append!(corr_pv_all_normalized_tog_control, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog_control, [density_shift])
        
        
    append!(corr_trials_all_exp_control, [corr_trials_all_earlylate[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp_control, [pv_map_normalized_earlylate])
    append!(peak_shift_mm_exp_control, [peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)]])
 
    
    
end





subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(0, 50,n_bins)
y_tog = peak_shift_mm_tog_control
y_min, y_max = plot_mean_density(bins, y_tog)
append!(y_min_all, y_min)    
append!(y_max_all, y_max)  



subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(0, 50,n_bins)
y_tog = peak_shift_mm_tog
y_min, y_max = plot_mean_density(bins, y_tog; plot_color="r", shade_color="r", line_color="r")
append!(y_min_all, y_min)    
append!(y_max_all, y_max)  
    
    
xlabel("PF shift (mm)", labelpad=-0.5)
    
title(" ", pad=20, rotation = 30)
if which_job == 1
    ylabel("Density")
end
    
if which_job != 1
    yticks([])
end
    
    
x = reduce(vcat, peak_shift_mm_exp)
y = reduce(vcat, peak_shift_mm_exp_control)

valid_neuron = isfinite.(x) .* isfinite.(y)
_,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "greater")
siginficance = "n.s."
if p_value < 0.01
    siginficance = "*"
end
if p_value < 1e-3
    siginficance = "**"
end
if p_value < 1e-5
    siginficance = "***"
end
println(p_value)


end


for which_job in 1:length(job_names_ordered)
    subplot(1,length(job_names_ordered),which_job)
    ylim(numpy.min([y_min_all]),numpy.max([y_max_all]))
end

fig.savefig(joinpath(fig_path, "peak_shift.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);

# original

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure6/summary/original"
mkpath(fig_path)

In [ ]:
fig = figure(figsize=(6,0.3))
y_min_all = []
y_max_all = []
for (which_job, job_name) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[which_job]
    
if job_name != "boundary_morphing"
    global compare_map_file_name = "compare_map_results_original.h5"
else
    global compare_map_file_name = "compare_map_results_original.h5"
end


n_bins = 31

corr_trials_all_tog = []
corr_pv_all_normalized_tog = []
peak_shift_mm_tog = []
    
corr_trials_all_exp = []
corr_pv_all_normalized_exp = []
peak_shift_mm_exp = []
    
corr_trials_all_tog_control = []
corr_pv_all_normalized_tog_control = []
peak_shift_mm_tog_control = []
    
corr_trials_all_exp_control = []
corr_pv_all_normalized_exp_control = []
peak_shift_mm_exp_control = []


for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    info_filename = joinpath(data_path(ds_save_cy_1), compare_map_file_name)
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late_original.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_earlylate = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all_earlylate = HDF5.readmmap(file["corr_earlylate_all"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm_earlylate = HDF5.readmmap(file["peak_shift_early_late_mm"])
    close(file)
        

    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]
    corr_pv_all_normalized_earlylate = pv_map_normalized_earlylate[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]

    density_corr_trials, bins = numpy.histogram(corr_trials_all[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)], bins= numpy.linspace(0, 50,n_bins), density=true)
 
    append!(corr_trials_all_tog, [density_corr_trials])
    append!(corr_pv_all_normalized_tog, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog, [density_shift])
        
        
    append!(corr_trials_all_exp, [corr_trials_all[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp, [pv_map_normalized])
    append!(peak_shift_mm_exp, [peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)]])
 

        
    density_corr_trials, bins = numpy.histogram(corr_trials_all_earlylate[tel_place_cell_index][isfinite.(corr_trials_all[tel_place_cell_index])], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized_earlylate, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)], bins= numpy.linspace(0, 50,n_bins), density=true)

    append!(corr_trials_all_tog_control, [density_corr_trials])
    append!(corr_pv_all_normalized_tog_control, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog_control, [density_shift])
        
        
    append!(corr_trials_all_exp_control, [corr_trials_all_earlylate[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp_control, [pv_map_normalized_earlylate])
    append!(peak_shift_mm_exp_control, [peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)]])
 
    
    
end





subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(-1,1,n_bins)
y_tog = corr_trials_all_tog_control
y_min, y_max = plot_mean_density(bins, y_tog)
append!(y_min_all, y_min)    
append!(y_max_all, y_max)  


subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(-1,1,n_bins)
y_tog = corr_trials_all_tog
y_min, y_max = plot_mean_density(bins, y_tog; plot_color="r", shade_color="r", line_color="r")
append!(y_min_all, y_min)    
append!(y_max_all, y_max)      

xlabel("PF correlation", labelpad = -0.5)
xlim(-1,1)
    
title(job_name, rotation = 30, fontsize = 5)
if which_job == 1
    ylabel("Density")
end
    
if which_job != 1
    yticks([])
end
 
x = reduce(vcat, corr_trials_all_exp)
y = reduce(vcat, corr_trials_all_exp_control)

valid_neuron = isfinite.(x) .* isfinite.(y)
_,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")
siginficance = "n.s."
if p_value < 0.01
    siginficance = "*"
end
if p_value < 1e-3
    siginficance = "**"
end
if p_value < 1e-5
    siginficance = "***"
end
println(p_value)



end

for which_job in 1:length(job_names_ordered)
    subplot(1,length(job_names_ordered),which_job)
    ylim(numpy.min([y_min_all]),numpy.max([y_max_all]))
end

fig.savefig(joinpath(fig_path, "PF_correlation.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);

In [ ]:
fig = figure(figsize=(6,0.3))
y_min_all = []
y_max_all = []
for (which_job, job_name) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[which_job]
    
if job_name != "boundary_morphing"
    global compare_map_file_name = "compare_map_results_original.h5"
else
    global compare_map_file_name = "compare_map_results_original.h5"
end


n_bins = 31

corr_trials_all_tog = []
corr_pv_all_normalized_tog = []
peak_shift_mm_tog = []
    
corr_trials_all_exp = []
corr_pv_all_normalized_exp = []
peak_shift_mm_exp = []
    
corr_trials_all_tog_control = []
corr_pv_all_normalized_tog_control = []
peak_shift_mm_tog_control = []
    
corr_trials_all_exp_control = []
corr_pv_all_normalized_exp_control = []
peak_shift_mm_exp_control = []


for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    info_filename = joinpath(data_path(ds_save_cy_1), compare_map_file_name)
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late_original.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_earlylate = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all_earlylate = HDF5.readmmap(file["corr_earlylate_all"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm_earlylate = HDF5.readmmap(file["peak_shift_early_late_mm"])
    close(file)
        

    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]
    corr_pv_all_normalized_earlylate = pv_map_normalized_earlylate[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]

    density_corr_trials, bins = numpy.histogram(corr_trials_all[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)], bins= numpy.linspace(0, 50,n_bins), density=true)
 
    append!(corr_trials_all_tog, [density_corr_trials])
    append!(corr_pv_all_normalized_tog, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog, [density_shift])
        
        
    append!(corr_trials_all_exp, [corr_trials_all[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp, [pv_map_normalized])
    append!(peak_shift_mm_exp, [peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)]])
 

        
    density_corr_trials, bins = numpy.histogram(corr_trials_all_earlylate[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized_earlylate, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)], bins= numpy.linspace(0, 50,n_bins), density=true)

    append!(corr_trials_all_tog_control, [density_corr_trials])
    append!(corr_pv_all_normalized_tog_control, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog_control, [density_shift])
        
        
    append!(corr_trials_all_exp_control, [corr_trials_all_earlylate[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp_control, [pv_map_normalized_earlylate])
    append!(peak_shift_mm_exp_control, [peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)]])
 

    
    
end





subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(-1,1,n_bins)
y_tog = corr_pv_all_normalized_tog_control
y_min, y_max = plot_mean_density(bins, y_tog)
append!(y_min_all, y_min)    
append!(y_max_all, y_max)  



subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(-1,1,n_bins)
y_tog = corr_pv_all_normalized_tog
y_min, y_max = plot_mean_density(bins, y_tog; plot_color="r", shade_color="r", line_color="r")
append!(y_min_all, y_min)    
append!(y_max_all, y_max)  
    
    
xlabel("PV correlation", labelpad=-0.5)
xlim(-1,1)
    
title(" ", pad=20, rotation = 30)
if which_job == 1
    ylabel("Density")
end
    
if which_job != 1
    yticks([])
end

    
    
x = reduce(vcat, corr_pv_all_normalized_exp)
y = reduce(vcat, corr_pv_all_normalized_exp_control)

valid_neuron = isfinite.(x) .* isfinite.(y)
_,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")
siginficance = "n.s."
if p_value < 0.01
    siginficance = "*"
end
if p_value < 1e-3
    siginficance = "**"
end
if p_value < 1e-5
    siginficance = "***"
end
println(p_value)



end

for which_job in 1:length(job_names_ordered)
    subplot(1,length(job_names_ordered),which_job)
    ylim(numpy.min([y_min_all]),numpy.max([y_max_all]))
end
fig.savefig(joinpath(fig_path, "PV_correlation.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);

In [ ]:
y_min_all = []
y_max_all = []

fig = figure(figsize=(6,0.3))

for (which_job, job_name) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[which_job]
    
if job_name != "boundary_morphing"
    global compare_map_file_name = "compare_map_results_original.h5"
else
    global compare_map_file_name = "compare_map_results_original.h5"
end


n_bins = 31

corr_trials_all_tog = []
corr_pv_all_normalized_tog = []
peak_shift_mm_tog = []
    
corr_trials_all_exp = []
corr_pv_all_normalized_exp = []
peak_shift_mm_exp = []
    
corr_trials_all_tog_control = []
corr_pv_all_normalized_tog_control = []
peak_shift_mm_tog_control = []
    
corr_trials_all_exp_control = []
corr_pv_all_normalized_exp_control = []
peak_shift_mm_exp_control = []


for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    info_filename = joinpath(data_path(ds_save_cy_1), compare_map_file_name)
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late_original.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_earlylate = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all_earlylate = HDF5.readmmap(file["corr_earlylate_all"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm_earlylate = HDF5.readmmap(file["peak_shift_early_late_mm"])
    close(file)
        

    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]
    corr_pv_all_normalized_earlylate = pv_map_normalized_earlylate[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]

    density_corr_trials, bins = numpy.histogram(corr_trials_all[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)], bins= numpy.linspace(0, 50,n_bins), density=true)
 
    append!(corr_trials_all_tog, [density_corr_trials])
    append!(corr_pv_all_normalized_tog, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog, [density_shift])
        
        
    append!(corr_trials_all_exp, [corr_trials_all[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp, [pv_map_normalized])
    append!(peak_shift_mm_exp, [peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)]])
 

        
    density_corr_trials, bins = numpy.histogram(corr_trials_all_earlylate[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized_earlylate, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)], bins= numpy.linspace(0, 50,n_bins), density=true)

    append!(corr_trials_all_tog_control, [density_corr_trials])
    append!(corr_pv_all_normalized_tog_control, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog_control, [density_shift])
        
        
    append!(corr_trials_all_exp_control, [corr_trials_all_earlylate[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp_control, [pv_map_normalized_earlylate])
    append!(peak_shift_mm_exp_control, [peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)]])
 
    
    
end





subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(0, 50,n_bins)
y_tog = peak_shift_mm_tog_control
y_min, y_max = plot_mean_density(bins, y_tog)
append!(y_min_all, y_min)    
append!(y_max_all, y_max)  



subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(0, 50,n_bins)
y_tog = peak_shift_mm_tog
y_min, y_max = plot_mean_density(bins, y_tog; plot_color="r", shade_color="r", line_color="r")
append!(y_min_all, y_min)    
append!(y_max_all, y_max)  
    
    
xlabel("PF shift (mm)", labelpad=-0.5)
    
title(" ", pad=20, rotation = 30)
if which_job == 1
    ylabel("Density")
end
    
if which_job != 1
    yticks([])
end
    
    
x = reduce(vcat, peak_shift_mm_exp)
y = reduce(vcat, peak_shift_mm_exp_control)

valid_neuron = isfinite.(x) .* isfinite.(y)
_,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "greater")
siginficance = "n.s."
if p_value < 0.01
    siginficance = "*"
end
if p_value < 1e-3
    siginficance = "**"
end
if p_value < 1e-5
    siginficance = "***"
end
println(p_value)


end


for which_job in 1:length(job_names_ordered)
    subplot(1,length(job_names_ordered),which_job)
    ylim(numpy.min([y_min_all]),numpy.max([y_max_all]))
end

fig.savefig(joinpath(fig_path, "peak_shift.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);

# best

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure6/summary/best"
mkpath(fig_path)

In [ ]:
fig = figure(figsize=(6,0.3))
y_min_all = []
y_max_all = []
for (which_job, job_name) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[which_job]
    
if job_name != "boundary_morphing"
    global compare_map_file_name = "compare_map_results.h5"
else
    global compare_map_file_name = "compare_map_results.h5"
end


n_bins = 31

corr_trials_all_tog = []
corr_pv_all_normalized_tog = []
peak_shift_mm_tog = []
    
corr_trials_all_exp = []
corr_pv_all_normalized_exp = []
peak_shift_mm_exp = []
    
corr_trials_all_tog_control = []
corr_pv_all_normalized_tog_control = []
peak_shift_mm_tog_control = []
    
corr_trials_all_exp_control = []
corr_pv_all_normalized_exp_control = []
peak_shift_mm_exp_control = []


for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    info_filename = joinpath(data_path(ds_save_cy_1), compare_map_file_name)
    file = h5open(info_filename, "r")
    confined_place_cell_index_1 = HDF5.readmmap(file["confined_place_cell_index_1"])
    confined_place_cell_index_2 = HDF5.readmmap(file["confined_place_cell_index_2"])
    confined_place_cell_index = intersect(confined_place_cell_index_1, confined_place_cell_index_2)
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late_best.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_earlylate = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all_earlylate = HDF5.readmmap(file["corr_earlylate_all"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm_earlylate = HDF5.readmmap(file["peak_shift_early_late_mm"])
    close(file)
        

    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]
    corr_pv_all_normalized_earlylate = pv_map_normalized_earlylate[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]

    density_corr_trials, bins = numpy.histogram(corr_trials_all[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)], bins= numpy.linspace(0, 50,n_bins), density=true)
 
    append!(corr_trials_all_tog, [density_corr_trials])
    append!(corr_pv_all_normalized_tog, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog, [density_shift])
        
        
    append!(corr_trials_all_exp, [corr_trials_all[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp, [pv_map_normalized])
    append!(peak_shift_mm_exp, [peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)]])
 

        
    density_corr_trials, bins = numpy.histogram(corr_trials_all_earlylate[tel_place_cell_index][isfinite.(corr_trials_all[tel_place_cell_index])], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized_earlylate, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)], bins= numpy.linspace(0, 50,n_bins), density=true)

    append!(corr_trials_all_tog_control, [density_corr_trials])
    append!(corr_pv_all_normalized_tog_control, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog_control, [density_shift])
        
        
    append!(corr_trials_all_exp_control, [corr_trials_all_earlylate[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp_control, [pv_map_normalized_earlylate])
    append!(peak_shift_mm_exp_control, [peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)]])
 

    
    
end





subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(-1,1,n_bins)
y_tog = corr_trials_all_tog_control
y_min, y_max = plot_mean_density(bins, y_tog)
append!(y_min_all, y_min)    
append!(y_max_all, y_max)  


subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(-1,1,n_bins)
y_tog = corr_trials_all_tog
y_min, y_max = plot_mean_density(bins, y_tog; plot_color="r", shade_color="r", line_color="r")
append!(y_min_all, y_min)    
append!(y_max_all, y_max)      

xlabel("PF correlation", labelpad = -0.5)
xlim(-1,1)
    
title(job_name, rotation = 30, fontsize = 5)
if which_job == 1
    ylabel("Density")
end
    
if which_job != 1
    yticks([])
end
 
x = reduce(vcat, corr_trials_all_exp)
y = reduce(vcat, corr_trials_all_exp_control)

valid_neuron = isfinite.(x) .* isfinite.(y)
_,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")
siginficance = "n.s."
if p_value < 0.01
    siginficance = "*"
end
if p_value < 1e-3
    siginficance = "**"
end
if p_value < 1e-5
    siginficance = "***"
end
println(p_value)



end

for which_job in 1:length(job_names_ordered)
    subplot(1,length(job_names_ordered),which_job)
    ylim(numpy.min([y_min_all]),numpy.max([y_max_all]))
end

fig.savefig(joinpath(fig_path, "PF_correlation.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);

In [ ]:
fig = figure(figsize=(6,0.3))
y_min_all = []
y_max_all = []
for (which_job, job_name) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[which_job]
    
if job_name != "boundary_morphing"
    global compare_map_file_name = "compare_map_results.h5"
else
    global compare_map_file_name = "compare_map_results.h5"
end


n_bins = 31

corr_trials_all_tog = []
corr_pv_all_normalized_tog = []
peak_shift_mm_tog = []
    
corr_trials_all_exp = []
corr_pv_all_normalized_exp = []
peak_shift_mm_exp = []
    
corr_trials_all_tog_control = []
corr_pv_all_normalized_tog_control = []
peak_shift_mm_tog_control = []
    
corr_trials_all_exp_control = []
corr_pv_all_normalized_exp_control = []
peak_shift_mm_exp_control = []


for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    info_filename = joinpath(data_path(ds_save_cy_1), compare_map_file_name)
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index_1 = HDF5.readmmap(file["confined_place_cell_index_1"])
    confined_place_cell_index_2 = HDF5.readmmap(file["confined_place_cell_index_2"])
    confined_place_cell_index = intersect(confined_place_cell_index_1, confined_place_cell_index_2)
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late_best.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_earlylate = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all_earlylate = HDF5.readmmap(file["corr_earlylate_all"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm_earlylate = HDF5.readmmap(file["peak_shift_early_late_mm"])
    close(file)
        

    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]
    corr_pv_all_normalized_earlylate = pv_map_normalized_earlylate[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]

    density_corr_trials, bins = numpy.histogram(corr_trials_all[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)], bins= numpy.linspace(0, 50,n_bins), density=true)
 
    append!(corr_trials_all_tog, [density_corr_trials])
    append!(corr_pv_all_normalized_tog, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog, [density_shift])
        
        
    append!(corr_trials_all_exp, [corr_trials_all[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp, [pv_map_normalized])
    append!(peak_shift_mm_exp, [peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)]])
 

        
    density_corr_trials, bins = numpy.histogram(corr_trials_all_earlylate[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized_earlylate, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)], bins= numpy.linspace(0, 50,n_bins), density=true)

    append!(corr_trials_all_tog_control, [density_corr_trials])
    append!(corr_pv_all_normalized_tog_control, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog_control, [density_shift])
        
        
    append!(corr_trials_all_exp_control, [corr_trials_all_earlylate[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp_control, [pv_map_normalized_earlylate])
    append!(peak_shift_mm_exp_control, [peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)]])
 
    
end





subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(-1,1,n_bins)
y_tog = corr_pv_all_normalized_tog_control
y_min, y_max = plot_mean_density(bins, y_tog)
append!(y_min_all, y_min)    
append!(y_max_all, y_max)  



subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(-1,1,n_bins)
y_tog = corr_pv_all_normalized_tog
y_min, y_max = plot_mean_density(bins, y_tog; plot_color="r", shade_color="r", line_color="r")
append!(y_min_all, y_min)    
append!(y_max_all, y_max)  
    
    
xlabel("PV correlation", labelpad=-0.5)
xlim(-1,1)
    
title(" ", pad=20, rotation = 30)
if which_job == 1
    ylabel("Density")
end
    
if which_job != 1
    yticks([])
end

    
    
x = reduce(vcat, corr_pv_all_normalized_exp)
y = reduce(vcat, corr_pv_all_normalized_exp_control)

valid_neuron = isfinite.(x) .* isfinite.(y)
_,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")
siginficance = "n.s."
if p_value < 0.01
    siginficance = "*"
end
if p_value < 1e-3
    siginficance = "**"
end
if p_value < 1e-5
    siginficance = "***"
end
println(p_value)



end

for which_job in 1:length(job_names_ordered)
    subplot(1,length(job_names_ordered),which_job)
    ylim(numpy.min([y_min_all]),numpy.max([y_max_all]))
end
fig.savefig(joinpath(fig_path, "PV_correlation.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);

In [ ]:
y_min_all = []
y_max_all = []

fig = figure(figsize=(6,0.3))

for (which_job, job_name) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[which_job]
    
if job_name != "boundary_morphing"
    global compare_map_file_name = "compare_map_results.h5"
else
    global compare_map_file_name = "compare_map_results.h5"
end


n_bins = 31

corr_trials_all_tog = []
corr_pv_all_normalized_tog = []
peak_shift_mm_tog = []
    
corr_trials_all_exp = []
corr_pv_all_normalized_exp = []
peak_shift_mm_exp = []
    
corr_trials_all_tog_control = []
corr_pv_all_normalized_tog_control = []
peak_shift_mm_tog_control = []
    
corr_trials_all_exp_control = []
corr_pv_all_normalized_exp_control = []
peak_shift_mm_exp_control = []


for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    info_filename = joinpath(data_path(ds_save_cy_1), compare_map_file_name)
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index_1 = HDF5.readmmap(file["confined_place_cell_index_1"])
    confined_place_cell_index_2 = HDF5.readmmap(file["confined_place_cell_index_2"])
    confined_place_cell_index = intersect(confined_place_cell_index_1, confined_place_cell_index_2)
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late_best.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_earlylate = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all_earlylate = HDF5.readmmap(file["corr_earlylate_all"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm_earlylate = HDF5.readmmap(file["peak_shift_early_late_mm"])
    close(file)
        

    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]
    corr_pv_all_normalized_earlylate = pv_map_normalized_earlylate[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]

    density_corr_trials, bins = numpy.histogram(corr_trials_all[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)], bins= numpy.linspace(0, 50,n_bins), density=true)
 
    append!(corr_trials_all_tog, [density_corr_trials])
    append!(corr_pv_all_normalized_tog, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog, [density_shift])
        
        
    append!(corr_trials_all_exp, [corr_trials_all[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp, [pv_map_normalized])
    append!(peak_shift_mm_exp, [peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)]])
 

        
    density_corr_trials, bins = numpy.histogram(corr_trials_all_earlylate[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized_earlylate, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)], bins= numpy.linspace(0, 50,n_bins), density=true)

    append!(corr_trials_all_tog_control, [density_corr_trials])
    append!(corr_pv_all_normalized_tog_control, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog_control, [density_shift])
        
        
    append!(corr_trials_all_exp_control, [corr_trials_all_earlylate[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp_control, [pv_map_normalized_earlylate])
    append!(peak_shift_mm_exp_control, [peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)]])
 
    
end





subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(0, 50,n_bins)
y_tog = peak_shift_mm_tog_control
y_min, y_max = plot_mean_density(bins, y_tog)
append!(y_min_all, y_min)    
append!(y_max_all, y_max)  



subplot(1,length(job_names_ordered),which_job)
bins = numpy.linspace(0, 50,n_bins)
y_tog = peak_shift_mm_tog
y_min, y_max = plot_mean_density(bins, y_tog; plot_color="r", shade_color="r", line_color="r")
append!(y_min_all, y_min)    
append!(y_max_all, y_max)  
    
    
xlabel("PF shift (mm)", labelpad=-0.5)
    
title(" ", pad=20, rotation = 30)
if which_job == 1
    ylabel("Density")
end
    
if which_job != 1
    yticks([])
end
    
    
x = reduce(vcat, peak_shift_mm_exp)
y = reduce(vcat, peak_shift_mm_exp_control)

valid_neuron = isfinite.(x) .* isfinite.(y)
_,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "greater")
siginficance = "n.s."
if p_value < 0.01
    siginficance = "*"
end
if p_value < 1e-3
    siginficance = "**"
end
if p_value < 1e-5
    siginficance = "***"
end
println(p_value)



end


for which_job in 1:length(job_names_ordered)
    subplot(1,length(job_names_ordered),which_job)
    ylim(numpy.min([y_min_all]),numpy.max([y_max_all]))
end

fig.savefig(joinpath(fig_path, "peak_shift.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);

# match angle to best

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure10/summary"
mkpath(fig_path)

In [ ]:
job_names_ordered

In [ ]:
job_names

In [ ]:
job_names_figure

In [ ]:
fig_path


In [ ]:
fig = figure(figsize=(1.8,1))
tick_locs = []
tick_labels = []
for (i_job, which_job) in enumerate([3, 7, 8])
data_info_all = data_info_dict[job_names_ordered[which_job]]
job_name = job_names_figure[which_job]

n_bins = 31

anglematch_median_tog = []
best_median_tog = []
    

for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
        

    append!(anglematch_median_tog, nanmedian(corr_trials_all[tel_place_cell_index]))

        
        
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
    


    
    append!(best_median_tog, nanmedian(corr_trials_all[tel_place_cell_index]))
    

    
    
end


for i in 1:length(best_median_tog)
    plot([i_job*2-1,i_job*2], [anglematch_median_tog[i], best_median_tog[i]], color="gray", alpha=0.5)
    scatter([i_job*2-1], anglematch_median_tog[i], color="blue", alpha=0.7, label = "Wall referred registration")
    scatter([i_job*2], best_median_tog[i], color="red", alpha=0.7, label = "Best wall referred registration")
        
end
    println(mean(best_median_tog .- anglematch_median_tog))
    println(std(best_median_tog .- anglematch_median_tog))


yticks([0,0.3,0.6])


end
xticks([])
ylabel("PF correlation")

fig.savefig(joinpath(fig_path, "PF_correlation_compare.pdf"), bbox_inches="tight",transparent = true);

In [ ]:
fig = figure(figsize=(1.8,1))
tick_locs = []
tick_labels = []
for (i_job, which_job) in enumerate([3, 7, 8])
data_info_all = data_info_dict[job_names_ordered[which_job]]
job_name = job_names_figure[which_job]

n_bins = 31

anglematch_median_tog = []
best_median_tog = []
    

for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized)]
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
        

    append!(anglematch_median_tog, nanmedian(corr_pv_all_normalized))

        
        
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized)]
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
    


    
    append!(best_median_tog, nanmedian(corr_pv_all_normalized))
    

    
    
end
println(anglematch_median_tog)
println(best_median_tog)


for i in 1:length(best_median_tog)
    plot([i_job*2-1,i_job*2], [anglematch_median_tog[i], best_median_tog[i]], color="gray", alpha=0.5)
        
    scatter([i_job*2-1], anglematch_median_tog[i], color="blue", alpha=0.7, label = "Wall referred registration")
    scatter([i_job*2], best_median_tog[i], color="red", alpha=0.7, label = "Best wall referred registration")

        
end
    println(mean(best_median_tog .- anglematch_median_tog))
    println(std(best_median_tog .- anglematch_median_tog))

    
yticks([0,0.3,0.6])

end

xticks(tick_locs, tick_labels, rotation=90)
ylabel("PV correlation")
fig.savefig(joinpath(fig_path, "PV_correlation_compare.pdf"), bbox_inches="tight",transparent = true);

In [ ]:
fig = figure(figsize=(1.8,1))
tick_locs = []
tick_labels = []
for (i_job, which_job) in enumerate([3, 7, 8])
data_info_all = data_info_dict[job_names_ordered[which_job]]
job_name = job_names_figure[which_job]

n_bins = 31

anglematch_median_tog = []
best_median_tog = []
    

for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
        

    append!(anglematch_median_tog, nanmedian(peak_shift_mm))

        
        
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
    


    
    append!(best_median_tog, nanmedian(peak_shift_mm))
    

    
    
end


for i in 1:length(best_median_tog)
    plot([i_job*2-1,i_job*2], [anglematch_median_tog[i], best_median_tog[i]], color="gray", alpha=0.5)
        
    scatter([i_job*2-1], anglematch_median_tog[i], color="blue", alpha=0.7, label = "Registration based on\nchamber wall geometry and landmarks")
    scatter([i_job*2], best_median_tog[i], color="red", alpha=0.7, label = "Nonrigid transformation\nwith the best rotation angle")
    if i == 1 && i_job == 1
        legend(frameon=false, loc=(1,0.2))
    end
        
end

    println(mean(best_median_tog .- anglematch_median_tog))
    println(std(best_median_tog .- anglematch_median_tog))


end

xticks(tick_locs, tick_labels, rotation=90)
ylabel("PF shift (mm)")

yticks([0,10,20])
fig.savefig(joinpath(fig_path, "peak_shift_compare.pdf"), bbox_inches="tight",transparent = true);

In [ ]:
fig_path